# Projeto SQL Corporativo
Este notebook demonstra o uso de SQL puro em um ambiente Python (DuckDB), com consultas analíticas e relacionamentos entre tabelas de uma empresa fictícia.

**Configuração**

In [13]:
!pip install jupysql duckdb-engine
%load_ext sql

%sql duckdb://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


**CRIAR TABELA**

Cria uma tabela chamada funcionarios para armazenar informações sobre os colaboradores de uma empresa.
Antes de criá-la, o comando DROP TABLE IF EXISTS remove a tabela caso ela já exista, evitando erros de duplicação.

A estrutura da tabela define os seguintes campos:

| Coluna          | Tipo de dado           | Descrição                                             |
| --------------- | ---------------------- | ----------------------------------------------------- |
| `id`            | `INT` (chave primária) | Identificador único de cada funcionário.              |
| `nome`          | `VARCHAR(100)`         | Nome completo do funcionário.                         |
| `cargo`         | `VARCHAR(50)`          | Função ou cargo ocupado na empresa.                   |
| `departamento`  | `VARCHAR(50)`          | Setor ou departamento ao qual o funcionário pertence. |
| `salario`       | `DECIMAL(10,2)`        | Salário do funcionário, com duas casas decimais.      |
| `cidade`        | `VARCHAR(50)`          | Cidade onde o funcionário trabalha.                   |
| `idade`         | `INT`                  | Idade atual do funcionário.                           |
| `data_admissao` | `DATE`                 | Data em que o funcionário foi contratado.             |

In [14]:
%%sql

CREATE TABLE IF NOT EXISTS departamentos (
    id INT PRIMARY KEY,
    nome VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS cidades (
    id INT PRIMARY KEY,
    nome VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS cargos (
    id INT PRIMARY KEY,
    nome VARCHAR(50),
    departamento_id INT,
    FOREIGN KEY (departamento_id) REFERENCES departamentos(id)
);

-- Tabela principal
CREATE TABLE IF NOT EXISTS funcionarios (
    id INT PRIMARY KEY,
    nome VARCHAR(100),
    cargo_id INT,
    cidade_id INT,
    salario DECIMAL(10,2),
    idade INT,
    data_admissao DATE,
    FOREIGN KEY (cargo_id) REFERENCES cargos(id),
    FOREIGN KEY (cidade_id) REFERENCES cidades(id)
);

Running query in 'duckdb://'

Count


**INSERIR DADOS**

In [15]:
%%sql

-- ================================
-- Limpar tabelas antigas (opcional)
-- ================================
DROP TABLE IF EXISTS funcionarios;
DROP TABLE IF EXISTS cargos;
DROP TABLE IF EXISTS cidades;
DROP TABLE IF EXISTS departamentos;

-- ================================
-- 1️⃣ Departamentos
-- ================================
CREATE TABLE IF NOT EXISTS departamentos (
    id INT PRIMARY KEY,
    nome VARCHAR(50)
);

INSERT INTO departamentos (id, nome) VALUES
(1, 'TI'),
(2, 'Marketing'),
(3, 'Financeiro');

-- ================================
-- 2️⃣ Cidades
-- ================================
CREATE TABLE IF NOT EXISTS cidades (
    id INT PRIMARY KEY,
    nome VARCHAR(50)
);

INSERT INTO cidades (id, nome) VALUES
(1, 'São Paulo'),
(2, 'Rio de Janeiro'),
(3, 'Curitiba'),
(4, 'Belo Horizonte'),
(5, 'Porto Alegre'),
(6, 'Recife');

-- ================================
-- 3️⃣ Cargos
-- ================================
CREATE TABLE IF NOT EXISTS cargos (
    id INT PRIMARY KEY,
    nome VARCHAR(50),
    departamento_id INT,
    FOREIGN KEY (departamento_id) REFERENCES departamentos(id)
);

INSERT INTO cargos (id, nome, departamento_id) VALUES
(1, 'Analista de Dados', 1),
(2, 'Desenvolvedor', 1),
(3, 'Gerente de Projetos', 1),
(4, 'Designer', 2),
(5, 'Coordenadora MKT', 2),
(6, 'Analista Financeira', 3),
(7, 'Estagiária', 3);

-- ================================
-- 4️⃣ Funcionários
-- ================================
CREATE TABLE IF NOT EXISTS funcionarios (
    id INT PRIMARY KEY,
    nome VARCHAR(100),
    cargo_id INT,
    cidade_id INT,
    salario DECIMAL(10,2),
    idade INT,
    data_admissao DATE,
    FOREIGN KEY (cargo_id) REFERENCES cargos(id),
    FOREIGN KEY (cidade_id) REFERENCES cidades(id)
);

INSERT INTO funcionarios (id, nome, cargo_id, cidade_id, salario, idade, data_admissao) VALUES
(1, 'Ana Souza', 1, 1, 6500, 29, '2020-05-10'),
(2, 'João Lima', 2, 2, 7200, 33, '2018-03-14'),
(3, 'Maria Santos', 4, 3, 4800, 26, '2021-09-01'),
(4, 'Pedro Oliveira', 3, 1, 9500, 41, '2015-01-22'),
(5, 'Júlia Martins', 6, 4, 5800, 30, '2019-06-18'),
(6, 'Lucas Almeida', 2, 5, 7100, 28, '2020-11-05'),
(7, 'Carla Dias', 5, 1, 8500, 35, '2016-02-27'),
(8, 'Rafael Gomes', 1, 3, 6700, 27, '2022-01-15'),
(9, 'Fernanda Costa', 7, 6, 2200, 22, '2023-03-10'),
(10, 'Bruno Ferreira', 2, 2, 7500, 31, '2017-04-30');

Running query in 'duckdb://'

Count


# Consultas sobre funcionários

**Listar todos os funcionários**

In [16]:
%%sql
SELECT * FROM funcionarios;

Running query in 'duckdb://'

id,nome,cargo_id,cidade_id,salario,idade,data_admissao
1,Ana Souza,1,1,6500.00,29,2020-05-10
2,João Lima,2,2,7200.00,33,2018-03-14
3,Maria Santos,4,3,4800.00,26,2021-09-01
4,Pedro Oliveira,3,1,9500.00,41,2015-01-22
5,Júlia Martins,6,4,5800.00,30,2019-06-18
6,Lucas Almeida,2,5,7100.00,28,2020-11-05
7,Carla Dias,5,1,8500.00,35,2016-02-27
8,Rafael Gomes,1,3,6700.00,27,2022-01-15
9,Fernanda Costa,7,6,2200.00,22,2023-03-10
10,Bruno Ferreira,2,2,7500.00,31,2017-04-30


**uncionários com idade entre 25 e 35**

In [17]:
%%sql
SELECT * FROM funcionarios WHERE idade BETWEEN 25 AND 35;

Running query in 'duckdb://'

id,nome,cargo_id,cidade_id,salario,idade,data_admissao
1,Ana Souza,1,1,6500.00,29,2020-05-10
2,João Lima,2,2,7200.00,33,2018-03-14
3,Maria Santos,4,3,4800.00,26,2021-09-01
5,Júlia Martins,6,4,5800.00,30,2019-06-18
6,Lucas Almeida,2,5,7100.00,28,2020-11-05
7,Carla Dias,5,1,8500.00,35,2016-02-27
8,Rafael Gomes,1,3,6700.00,27,2022-01-15
10,Bruno Ferreira,2,2,7500.00,31,2017-04-30


**Funcionários admitidos após 2020**

In [18]:
%%sql
SELECT * FROM funcionarios WHERE data_admissao > '2020-12-31';

Running query in 'duckdb://'

id,nome,cargo_id,cidade_id,salario,idade,data_admissao
3,Maria Santos,4,3,4800.00,26,2021-09-01
8,Rafael Gomes,1,3,6700.00,27,2022-01-15
9,Fernanda Costa,7,6,2200.00,22,2023-03-10


**Top 3 maiores salários**

In [19]:
%%sql
SELECT * FROM funcionarios ORDER BY salario DESC LIMIT 3;

Running query in 'duckdb://'

id,nome,cargo_id,cidade_id,salario,idade,data_admissao
4,Pedro Oliveira,3,1,9500.00,41,2015-01-22
7,Carla Dias,5,1,8500.00,35,2016-02-27
10,Bruno Ferreira,2,2,7500.00,31,2017-04-30


# Consultas sobre Departamentos

**Listar todos os departamentos**

In [20]:
%%sql
SELECT * FROM departamentos;

Running query in 'duckdb://'

id,nome
1,TI
2,Marketing
3,Financeiro


**Média salarial por departamento**

In [22]:
%%sql
SELECT d.nome AS departamento, AVG(f.salario) AS media_salarial
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
JOIN departamentos d ON c.departamento_id = d.id
GROUP BY d.nome;

Running query in 'duckdb://'

departamento,media_salarial
TI,7416.666666666667
Marketing,6650.0
Financeiro,4000.0


**Funcionários do departamento de TI**

In [24]:
%%sql
SELECT f.nome, c.nome AS cargo
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
JOIN departamentos d ON c.departamento_id = d.id
WHERE d.nome = 'TI';

Running query in 'duckdb://'

nome,cargo
Ana Souza,Analista de Dados
João Lima,Desenvolvedor
Pedro Oliveira,Gerente de Projetos
Lucas Almeida,Desenvolvedor
Rafael Gomes,Analista de Dados
Bruno Ferreira,Desenvolvedor


**Funcionários cujo cargo contém "Analista"**

In [26]:
%%sql
SELECT f.*
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
WHERE c.nome LIKE '%Analista%';

Running query in 'duckdb://'

id,nome,cargo_id,cidade_id,salario,idade,data_admissao
1,Ana Souza,1,1,6500.00,29,2020-05-10
5,Júlia Martins,6,4,5800.00,30,2019-06-18
8,Rafael Gomes,1,3,6700.00,27,2022-01-15


**Atualizar salários do departamento de TI para 7000**

In [28]:
%%sql
UPDATE funcionarios f
SET salario = 7000
FROM cargos c
WHERE f.cargo_id = c.id
  AND c.departamento_id = 1;

Running query in 'duckdb://'

Count


# Consulta sobre localidades

**Listar todas as cidades**

In [29]:
%%sql
SELECT * FROM cidades;

Running query in 'duckdb://'

id,nome
1,São Paulo
2,Rio de Janeiro
3,Curitiba
4,Belo Horizonte
5,Porto Alegre
6,Recife


**Nomes distintos das cidades**

In [30]:
%%sql
SELECT DISTINCT nome FROM cidades;

Running query in 'duckdb://'

nome
São Paulo
Rio de Janeiro
Curitiba
Belo Horizonte
Recife
Porto Alegre


**Funcionários que trabalham em São Paulo ou Curitiba**

In [32]:
%%sql
SELECT f.nome, ci.nome AS cidade
FROM funcionarios f
JOIN cidades ci ON f.cidade_id = ci.id
WHERE ci.nome IN ('São Paulo', 'Curitiba');

Running query in 'duckdb://'

nome,cidade
Ana Souza,São Paulo
Maria Santos,Curitiba
Pedro Oliveira,São Paulo
Carla Dias,São Paulo
Rafael Gomes,Curitiba


**Média salarial por cidade**

In [34]:
%%sql
SELECT ci.nome AS cidade, AVG(f.salario) AS media_salarial
FROM funcionarios f
JOIN cidades ci ON f.cidade_id = ci.id
GROUP BY ci.nome;

Running query in 'duckdb://'

cidade,media_salarial
São Paulo,7500.0
Rio de Janeiro,7000.0
Curitiba,5900.0
Belo Horizonte,5800.0
Recife,2200.0
Porto Alegre,7000.0


**Contagem de funcionários por cidade**

In [35]:
%%sql
SELECT ci.nome AS cidade, COUNT(*) AS total_funcionarios
FROM funcionarios f
JOIN cidades ci ON f.cidade_id = ci.id
GROUP BY ci.nome;

Running query in 'duckdb://'

cidade,total_funcionarios
São Paulo,3
Rio de Janeiro,2
Curitiba,2
Belo Horizonte,1
Recife,1
Porto Alegre,1
